<a href="https://colab.research.google.com/github/palakharinkhede4/Daily-P-L-And-VaR-Calculator/blob/main/Daily_PL_And_VaR_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Step 1: Import necessary libraries
import requests
import pandas as pd
import numpy as np
from google.colab import userdata # To access secrets in Colab

# Step 2: Set up API Key and Stock Ticker
api_key = userdata.get('ALPHA_VANTAGE_KEY')
stock_ticker = 'AAPL' # Using Apple Inc. as an example

# Step 3: Fetch Data from Alpha Vantage API
print(f"Fetching daily price data for {stock_ticker}...")

# CHANGE 1: Switched from TIME_SERIES_DAILY_ADJUSTED to the free TIME_SERIES_DAILY endpoint.
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_ticker}&outputsize=full&apikey={api_key}'

try:
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()

    if "Time Series (Daily)" not in data:
        raise ValueError(f"Could not retrieve time series data. API response: {data}")

    df = pd.DataFrame(data['Time Series (Daily)']).T
    print("Data fetched successfully! ✅")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")
    raise

# Step 4: Data Processing and Cleaning
# We no longer need 'adjusted_close'
df.rename(columns={
    '1. open': 'open',
    '2. high': 'high',
    '3. low': 'low',
    '4. close': 'close',
    '5. volume': 'volume'
}, inplace=True)

# Convert all columns to numeric
for col in df.columns:
    df[col] = pd.to_numeric(df[col])

df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)

print("\nSample of the cleaned data:")
print(df.tail())

# Step 5: Calculate Daily P&L (as percentage return)
# CHANGE 2: Use the 'close' column for calculations.
df['daily_pl'] = df['close'].pct_change() * 100 # In percentage terms

# Step 6: Calculate 0.95 Historical Value at Risk (VaR)
var_95 = np.percentile(df['daily_pl'].dropna(), 5)

# Step 7: Display Results
print("\n--- Risk Metrics Report ---")
last_pl = df['daily_pl'].iloc[-1]
last_date = df.index[-1].strftime('%Y-%m-%d')

print(f"📊 Daily P&L for {last_date}: {last_pl:.2f}%")
print(f"🔥 95% Historical VaR: {var_95:.2f}%")
print(f"\nInterpretation: Based on historical data, we are 95% confident that the maximum loss for {stock_ticker} on any given day will not exceed {-var_95:.2f}%.")

Fetching daily price data for AAPL...
Data fetched successfully! ✅

Sample of the cleaned data:
               open     high     low   close    volume
2025-07-24  213.900  215.690  213.53  213.76  46022620
2025-07-25  214.700  215.240  213.40  213.88  40268781
2025-07-28  214.030  214.845  213.06  214.05  37858017
2025-07-29  214.175  214.810  210.82  211.27  51411723
2025-07-30  211.895  212.390  207.72  209.05  45512514

--- Risk Metrics Report ---
📊 Daily P&L for 2025-07-30: -1.05%
🔥 95% Historical VaR: -3.63%

Interpretation: Based on historical data, we are 95% confident that the maximum loss for AAPL on any given day will not exceed 3.63%.
